Simply trying on Italy(Generation)

In [11]:
import logging
from itertools import combinations
import pandas as pd
import numpy as np
import random
from geopy.distance import geodesic
import networkx as nx
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

In [12]:
#Read cities data and distance matrix

CITIES = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
DIST_MATRIX = np.zeros((len(CITIES), len(CITIES)))
for c1, c2 in combinations(CITIES.itertuples(), 2):
    DIST_MATRIX[c1.Index, c2.Index] = DIST_MATRIX[c2.Index, c1.Index] = geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km
CITIES.head()

,name,lat,lon
0,Ancona,43.60,13.50
1,Andria,41.23,16.29
2,Bari,41.12,16.87
3,Bergamo,45.70,9.67
4,Bologna,44.50,11.34


In [13]:
#set parameters
population_size = 100
num_generations = 500
mutation_rate = 0.01

In [14]:
#initial population
def init_populations(size):
    population = []
    for _ in range(size):
        individual = list(range(len(CITIES)))
        random.shuffle(individual)
        population.append(individual)
    return population

In [15]:
#try to calculate fitness for better child
def fitness(individual):
    distance = sum(DIST_MATRIX[individual[i]][individual[i + 1]] for i in range(len(individual) - 1))
    distance += DIST_MATRIX[individual[-1]][individual[0]]  # return to start point
    return 1 / distance 
    #Taking the reciprocal is a way to make sure the shorter path correspond to a higher fitness values

In [16]:
def selection(population):
    
    fitness_values = [fitness(individual) for individual in population]
    total_fitness = sum(fitness_values)

    if total_fitness == 0:
            return population

    probabilities = [f / total_fitness for f in fitness_values]#so,the better child can get higher probalility
    selected_indices = np.random.choice(len(population), size=len(population), p=probabilities)
    selected = [population[i] for i in selected_indices]  # return individual ,or get a error 'a must be 1-dimensional '
    return selected

In [17]:
# crossover(I have to say,its a cool skill in basketball)
def crossover(parent1, parent2):
    start, end = sorted(random.sample(range(len(CITIES)), 2))
    child = [-1] * len(CITIES)#A child list and -1 means its not filling
    child[start:end] = parent1[start:end]
    
    # filling rest
    current_pos = end
    for city in parent2:
        if city not in child:
            child[current_pos] = city
            current_pos = (current_pos + 1) % len(CITIES)#back to start
            
    return child


In [18]:
def mutate(individual):
    for i in range(len(CITIES)):
        if random.random() < mutation_rate:#[0,1)
            j = random.randint(0, len(CITIES) - 1)
            individual[i], individual[j] = individual[j], individual[i]

In [19]:
def genetic_algorithm():
    
    population = init_populations(population_size)
    
    for generation in range(num_generations):
        population = selection(population)
        next_generation = []

        for i in range(0, population_size, 2):
            parent1 = population[i]
            parent2 = population[i + 1] if i + 1 < population_size else population[0]
            child = crossover(parent1, parent2)
            mutate(child)
            next_generation.append(child)

        population = next_generation


    best_individual = min(population, key=lambda ind: 1 / fitness(ind))
    best_distance = 1 / fitness(best_individual)
    
    return best_individual, best_distance



In [20]:
best_path, best_distance = genetic_algorithm()

best_path_cities = CITIES.iloc[best_path]['name'].tolist()
print("最佳路径:", best_path_cities)
print("最佳距离:", best_distance)

IndexError: list index out of range